In [530]:
%load_ext autoreload
%autoreload 2
import numpy as np
import data
from models import TestModels
import moveFiles
from keras.models import load_model, Model
import keras
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [67]:
file_limit = 20
train, test, filenames = moveFiles.moveFiles(limit_files=file_limit)

pp = data.Preprocessing()
pp.extractAllVideos()

In [68]:
dataset = data.DataSet()
dataset.dumpNumpyFiles( seq_len_limit=20)

  0%|          | 0/13 [00:00<?, ?it/s]

Exporting Train Data...


  0%|          | 0/4 [00:00<?, ?it/s]

Exporting Test Data


100%|██████████| 4/4 [00:02<00:00,  1.44it/s]


In [69]:
x,y,yseq = dataset.all_data_from_npz('train')

In [555]:
tm = TestModels(5, 'lrcn')

In [556]:
tm.model.fit(x,y, epochs=50, shuffle=True)

Epoch 1/50
13/13 [==============================] - 10s 732ms/step - loss: 1.5490 - acc: 0.4615
Epoch 2/50
13/13 [==============================] - 8s 648ms/step - loss: 1.5289 - acc: 0.3846
Epoch 3/50
13/13 [==============================] - 8s 585ms/step - loss: 1.4064 - acc: 0.5385
Epoch 4/50
13/13 [==============================] - 7s 561ms/step - loss: 1.4565 - acc: 0.3846
Epoch 5/50
13/13 [==============================] - 7s 535ms/step - loss: 1.4637 - acc: 0.4615
Epoch 6/50
13/13 [==============================] - 7s 525ms/step - loss: 1.4240 - acc: 0.5385
Epoch 7/50
13/13 [==============================] - 7s 576ms/step - loss: 1.4950 - acc: 0.3846
Epoch 8/50
13/13 [==============================] - 7s 522ms/step - loss: 1.3586 - acc: 0.4615
Epoch 9/50
13/13 [==============================] - 7s 507ms/step - loss: 1.3322 - acc: 0.5385
Epoch 10/50
13/13 [==============================] - 7s 503ms/step - loss: 1.4111 - acc: 0.4615
Epoch 11/50
13/13 [=============================

In [557]:
tm.model.save('test-model.hdf5')
m = load_model('test-model.hdf5')

# Create Model to Pull 'Dense' Features

In [558]:
dense_m = Model(inputs=m.input, outputs=m.get_layer('fc1').output)

## Convert Data into the Format for LSTM Model

In [559]:
x_lstm = [np.array([x[i]]) for i in range(len(x))]
x_lstm = np.array([dense_m.predict(k) for k in x_lstm])
x_lstm = x_lstm.squeeze(axis=1) #squeeze to get into the right shape for LSTM
y_lstm = np.array([np.array([yseq[i]]) for i in range(len(yseq))])
y_lstm = y_lstm.squeeze(axis=1)

In [563]:
# lstm = TestModels(5, 'lstm')
lstm.model.fit(x_lstm, y_lstm, epochs=200, batch_size=1)

Epoch 1/200
13/13 [==============================] - 0s 15ms/step - loss: 0.8151 - acc: 0.8115
Epoch 2/200
13/13 [==============================] - 0s 15ms/step - loss: 0.8022 - acc: 0.8154
Epoch 3/200
13/13 [==============================] - 0s 14ms/step - loss: 0.7943 - acc: 0.8269
Epoch 4/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7943 - acc: 0.8154
Epoch 5/200
13/13 [==============================] - 0s 15ms/step - loss: 0.8158 - acc: 0.8154
Epoch 6/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7830 - acc: 0.8231
Epoch 7/200
13/13 [==============================] - 0s 16ms/step - loss: 0.8041 - acc: 0.8154
Epoch 8/200
13/13 [==============================] - 0s 15ms/step - loss: 0.8261 - acc: 0.8115
Epoch 9/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7947 - acc: 0.8231
Epoch 10/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7823 - acc: 0.8231
Epoch 11/200
13/13 [=============================

13/13 [==============================] - 0s 14ms/step - loss: 0.7376 - acc: 0.8154
Epoch 87/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7306 - acc: 0.8308
Epoch 88/200
13/13 [==============================] - 0s 14ms/step - loss: 0.7611 - acc: 0.8192
Epoch 89/200
13/13 [==============================] - 0s 14ms/step - loss: 0.7266 - acc: 0.8269
Epoch 90/200
13/13 [==============================] - 0s 16ms/step - loss: 0.7334 - acc: 0.8231
Epoch 91/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7113 - acc: 0.8269
Epoch 92/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7247 - acc: 0.8231
Epoch 93/200
13/13 [==============================] - 0s 14ms/step - loss: 0.7150 - acc: 0.8115
Epoch 94/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7255 - acc: 0.8269
Epoch 95/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7222 - acc: 0.8269
Epoch 96/200
13/13 [==============================] -

13/13 [==============================] - 0s 16ms/step - loss: 0.6896 - acc: 0.8385
Epoch 171/200
13/13 [==============================] - 0s 14ms/step - loss: 0.6843 - acc: 0.8423
Epoch 172/200
13/13 [==============================] - 0s 15ms/step - loss: 0.6656 - acc: 0.8385
Epoch 173/200
13/13 [==============================] - 0s 14ms/step - loss: 0.6809 - acc: 0.8423
Epoch 174/200
13/13 [==============================] - 0s 14ms/step - loss: 0.6529 - acc: 0.8308
Epoch 175/200
13/13 [==============================] - 0s 15ms/step - loss: 0.7086 - acc: 0.8346
Epoch 176/200
13/13 [==============================] - 0s 14ms/step - loss: 0.6702 - acc: 0.8308
Epoch 177/200
13/13 [==============================] - 0s 14ms/step - loss: 0.6520 - acc: 0.8423
Epoch 178/200
13/13 [==============================] - 0s 15ms/step - loss: 0.6525 - acc: 0.8462
Epoch 179/200
13/13 [==============================] - 0s 15ms/step - loss: 0.6800 - acc: 0.8269
Epoch 180/200
13/13 [=======================

In [523]:
preds

array([[[0.15772662, 0.22016934, 0.21478514, 0.22600219, 0.18131666],
        [0.13737012, 0.24336159, 0.23757575, 0.22889659, 0.15279596],
        [0.14084533, 0.26827225, 0.2349611 , 0.2277287 , 0.12819263],
        [0.1336434 , 0.2648916 , 0.23738754, 0.2371517 , 0.12692577],
        [0.12384777, 0.25966668, 0.2464062 , 0.23897444, 0.13110504],
        [0.1216127 , 0.27621013, 0.25931722, 0.21729437, 0.12556559],
        [0.12350351, 0.30517232, 0.25629488, 0.20074528, 0.11428399],
        [0.17719302, 0.36703503, 0.20810172, 0.16570716, 0.081963  ],
        [0.15881833, 0.34934646, 0.20629573, 0.1782313 , 0.1073081 ],
        [0.13887618, 0.36800525, 0.22086538, 0.17095467, 0.10129861],
        [0.13666183, 0.3827532 , 0.21830082, 0.16733171, 0.0949525 ],
        [0.13160451, 0.39341813, 0.21615763, 0.16510563, 0.09371403],
        [0.12405361, 0.39553115, 0.21653631, 0.1624374 , 0.1014416 ],
        [0.10604483, 0.3785264 , 0.2427527 , 0.16279788, 0.10987824],
        [0.10200866,

In [602]:
#Takes LSTM predictions and generates sequences by assuming all 'normal' unless there is a prediction with a
#value higher than threshold. 
def seqFromPreds(preds, threshold=0.35):
    preds = preds.squeeze()
    seq = ['normal']*20
    
    for i in range(len(preds)):
        p = preds[i]
        m = np.max(p)
        a = np.argmax(p)
        if m>=threshold:
            seq[i]=dataset.classes[a]
    return seq
    

In [605]:
n = 1
x1 = np.array([x[n]])
x1 = dense_m.predict(x1)
y1 = np.array([yseq[n]])

In [606]:
preds = lstm.model.predict(x1)
predSeq = seqFromPreds(preds)
actualSeq = [dataset.reverse_one_hot(k) for k in y1.squeeze()]
df = pd.DataFrame({'actual': actualSeq, 'predicted': predSeq})
df

,actual,predicted
0,normal,normal
1,normal,normal
2,normal,normal
3,normal,normal
4,normal,normal
5,normal,normal
6,normal,normal
7,normal,normal
8,normal,normal
9,normal,normal
